# Using tapispy to Execute Actor Containers on the Abaco Cloud

In this notebook, we explore the `tapispy` library's support for executing actor containers on the TACC Abaco Cloud. This tutorial assumes the user has a TACC account and access to the TACC Cloud APIs via a set of OAuth API keys. Full details on getting the required accounts can be found on Abaco's Getting Started Guide here: https://abaco.readthedocs.io/en/latest/getting-started/index.html#account-creation-and-software-installation

## Installation

The `tapispy` library is a Python library for interacting with TACC's Tapis API platform. It can be installed directly from PyPI using pip - `pip install tapispy`.


## Importing and Instantiating the Tapis object

The primary functionality is accessed through a Tapis client, a Python object containing methods for accessing the primary APIs within the Tapis framework. The first step is to import and instantiate one of these objects.

When instantiating the `Tapis` object, we must provide the `api_server` to interact with. This is because Tapis is a multi-tenant platform and serves different projects at different base URLs. To interact with the primary "TACC" tenant, use `api_server=https://api.tacc.utexas.edu`.

We also need to supply the `Tapis` constructor with some authentication credentials. Here we have several options, the simplest of which is to just provide an OAuth access token via the `token` parameter. You can also provide a TACC `username` and `password`, together with an `api_key` and `api_secret` and use the `Tapis` object to generate an access token. For now, we'll just assume we have an access token already.

In [3]:
from tapispy.tapis import Tapis
tp = Tapis(api_server='https://api.tacc.utexas.edu', token='<insert_access_token>')

## Listing Actors

The `tp` object has object members representing the various Tapis APIs. For instance, there is an `actors` object for interacting with the TACC Abaco APIs. The `actors` object has methods for interacting with specific endpoints within the Abaco API. For example, we can list actors using the following:

In [4]:
tp.actors.list()

[{'_links': {'executions': 'https://api.tacc.utexas.edu/actors/v2/YmDqmbXyRAj5D/executions',
   'owner': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs',
   'self': 'https://api.tacc.utexas.edu/actors/v2/YmDqmbXyRAj5D'},
  'createTime': '2019-06-18 16:35:39.817858',
  'defaultEnvironment': {},
  'description': '',
  'gid': 811324,
  'id': 'YmDqmbXyRAj5D',
  'image': 'abacosamples/test',
  'lastUpdateTime': '2019-06-18 16:35:39.817858',
  'mounts': [],
  'owner': 'jstubbs',
  'privileged': False,
  'queue': 'default',
  'state': {},
  'stateless': True,
  'status': 'READY',
  'statusMessage': ' ',
  'tasdir': '01837/jstubbs',
  'type': 'none',
  'uid': 811324,
  'useContainerUid': False},
 {'_links': {'executions': 'https://api.tacc.utexas.edu/actors/v2/6PlMbDLa4zlON/executions',
   'owner': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs',
   'self': 'https://api.tacc.utexas.edu/actors/v2/6PlMbDLa4zlON'},
  'createTime': '2018-07-03 23:14:55.433342',
  'defaultEnvironment': {},
  'd

We should see our `taccsciapps/abaco_classifier` actor in the list above. We can take note of the actor ID in a variable for use later:

In [5]:
# update this cell with your actor ID:
actor_id = 'pm0vLq1aEJW7'

## Executing the Classifier Actor

We are now ready to execute our actor. We do this by sending the actor a message which can be done using the `tp.actors.sendMessage()` method. We pass the actor id using the `actorId` parameter and the message data using the `body` parameter, as shown below:

In [6]:
image_url = 'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg'
tp.actors.sendMessage(actorId=actor_id, body={'message': image_url})

{'_links': {'messages': 'https://api.tacc.utexas.edu/actors/v2/pm0vLq1aEJW7/messages',
  'owner': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs',
  'self': 'https://api.tacc.utexas.edu/actors/v2/pm0vLq1aEJW7/executions/YyOJBgy10lWw8'},
 'executionId': 'YyOJBgy10lWw8',
 'msg': 'https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg'}

## Retrieving the Execution Details and Logs

Abaco responds with the `executionId` associated with our execution. We can use this to retrieve the execution details (including its status) and the logs associated with the execution. We use the `tp.actors.getExecution()` and `tp.actors.getExecutionLogs()` methods, respectively, as shown below:

In [8]:
# update with your execution id:
execution_id = 'YyOJBgy10lWw8'
ex_details = tp.actors.getExecution(actorId=actor_id, executionId=execution_id)
ex_details

{'_links': {'logs': 'https://api.tacc.utexas.edu/actors/v2/TACC-PROD_pm0vLq1aEJW7/executions/YyOJBgy10lWw8/logs',
  'owner': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs',
  'self': 'https://api.tacc.utexas.edu/actors/v2/TACC-PROD_pm0vLq1aEJW7/executions/YyOJBgy10lWw8'},
 'actorId': 'pm0vLq1aEJW7',
 'apiServer': 'https://api.tacc.utexas.edu',
 'cpu': 25684504485,
 'executor': 'jstubbs',
 'exitCode': 0,
 'finalState': {'Dead': False,
  'Error': '',
  'ExitCode': 0,
  'FinishedAt': '2019-07-14T20:28:57.870510748Z',
  'OOMKilled': False,
  'Paused': False,
  'Pid': 0,
  'Restarting': False,
  'Running': False,
  'StartedAt': '2019-07-14T20:28:49.891162864Z',
  'Status': 'exited'},
 'id': 'YyOJBgy10lWw8',
 'io': 446918670,
 'messageReceivedTime': '2019-07-14 20:28:42.531652',
 'runtime': 8,
 'startTime': '2019-07-14 20:28:49.228643',
 'status': 'COMPLETE',
 'workerId': 'raoPzaXMV7mwA'}

In [9]:
ex_logs = tp.actors.getExecutionLogs(actorId=actor_id, executionId=execution_id)
ex_logs

{'_links': {'execution': 'https://api.tacc.utexas.edu/actors/v2/pm0vLq1aEJW7/executions/YyOJBgy10lWw8',
  'owner': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs',
  'self': 'https://api.tacc.utexas.edu/actors/v2/pm0vLq1aEJW7/executions/YyOJBgy10lWw8/logs'},
 'logs': 'Contents of MSG: https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12231410/Labrador-Retriever-On-White-01.jpg\n2019-07-14 20:28:54.762906: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX\n2019-07-14 20:28:55.219176: W tensorflow/core/framework/op_def_util.cc:343] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().\n/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `

We see from the above that our classifier image classified the doa as a Lobrador retrieve with 97.47% certainty. Pretty good!

## Conclusion

We have seen how to execute an actors on the Abaco cloud using the `tapispy` library, but we have only scratched the surface of what is capable with the Abaco API. In subsequent tutorials, we will explore additional topics such as sending messages of different content types, including JSON and binary data, permissions and sharing actors with other researchers, actor events and building complex networks of actors, and more. Details on all of these topics can also be found on Abaco's official documentation site on ReadTheDocs: https://abaco.readthedocs.io